In [3]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv
import datetime

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
# w = open('../data/db_connect_data_rnn.csv', 'w', encoding='utf-8')
w = open('../data/db_connect_test_data_rnn.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

data_length = []

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# 데이터는 cct, swr, uvb, uvi 순
for j in range(len(date)):
    sql = "select time(date), date(date), cct, cas_swr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j][0]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
    # 일출 후 6시간과 일몰 후 6시간 데이터 사용, 일출 혹은 일몰 당시 데이터가 없을 경우 가장 가까운 다른 데이터로 변환
    sunrise = datetime.datetime.strptime(date[j][1], '%H:%M:%S')
    sunset = datetime.datetime.strptime(date[j][2], '%H:%M:%S')
    
    standard = datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    sunrise = sunrise - standard
    sunset = sunset - standard
    
    start = 0
    end = 0

    for i in range(len(rows)):
        if(rows[i][0] >= sunrise):
            start = i
            print(date[j][0], rows[start][0], sunrise)
            break;
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunset):
            end = i
            print(date[j][0], rows[end][0], sunset)
            break;
    
    last = int((end - start) / 4)
    last = last * 4
    
#     # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start, start + last):
        wr.writerow([rows[l][2], rows[l][3]])
    print(date[j][0] + " " + str(last) + " data saved.")
    for l in range(4):
        data_length.append(int(last/4))
    print('----------------------------------------------')
        
w.close()
db.close()

2018-04-19 5:52:00 5:52:00
2018-04-19 19:52:00 19:52:00
2018-04-19 832 data saved.
----------------------------------------------
2018-04-21 5:49:00 5:49:00
2018-04-21 19:49:00 19:49:00
2018-04-21 832 data saved.
----------------------------------------------
2018-04-26 5:43:00 5:43:00
2018-04-26 19:43:00 19:43:00
2018-04-26 832 data saved.
----------------------------------------------
2018-04-27 5:42:00 5:42:00
2018-04-27 19:42:00 19:42:00
2018-04-27 832 data saved.
----------------------------------------------
2018-04-28 5:41:00 5:41:00
2018-04-28 19:41:00 19:41:00
2018-04-28 832 data saved.
----------------------------------------------
2018-04-29 5:40:00 5:40:00
2018-04-29 19:40:00 19:40:00
2018-04-29 832 data saved.
----------------------------------------------
2018-04-30 5:38:00 5:38:00
2018-04-30 19:38:00 19:38:00
2018-04-30 832 data saved.
----------------------------------------------
2018-05-04 5:34:00 5:34:00
2018-05-04 19:34:00 19:34:00
2018-05-04 832 data saved.
-------

In [4]:
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# 데이터 받아오고 편차 계산히기
# import_data = np.loadtxt('../data/db_connect_data_rnn.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data_rnn.csv', delimiter=',')

one = import_data[:,0] # cct
two = import_data[:,1] # cas_swr

data_index = 0

cct = []
swr = []

delta_cct = []
delta_swr = []

cct_hap = []
swr_hap = []

for i in range(len(data_length)):
    temp = []
    temp2 = []
    for j in range(data_length[i]):
        temp.append(one[j + data_index])
        temp2.append(two[j + data_index])
    cct.append(temp)
    swr.append(temp2)
    data_index += data_length[i]

for i in range(len(data_length)):
    temp = []
    temp2 = []
    for j in range(data_length[i] - 1):
        temp.append(cct[i][j+1] - cct[i][j])
        temp2.append(swr[i][j+1] - swr[i][j])
    delta_cct.append(temp)
    delta_swr.append(temp2)
    
for i in range(len(data_length)):
    temp1 = 0
    temp2 = 0
    for j in range(data_length[i]):
        temp1 += cct[i][j]
        temp2 += swr[i][j]
    cct_hap.append(temp1)
    swr_hap.append(temp2)

print(np.shape(cct))

(148,)


In [3]:
# w = open('../data/cct_swr_calculation_rnn.csv', 'w', encoding='utf-8')
w = open('../data/cct_swr_calculation_test_rnn.csv', 'w', encoding='utf-8')

wr = csv.writer(w)

for i in range(len(delta_cct)):
    up_dot_25 = 0
    up_dot_50 = 0
    up_dot_100 = 0
    up_dot_150 = 0
    up_dot_200 = 0
    up_dot_250 = 0
    up_dot_300 = 0
    up_dot_350 = 0
    up_dot_400 = 0
    up_dot_450 = 0
    up_dot_500 = 0

    # 위 변수들을 비율로 계산
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_11 = 0

    # 편차의 합, 평균, 등급 저장
    hap = 0
    avg = 0
    grade = 0

    # 편차 절댓값을 기준으로 갯수 저장
    for j in range(len(delta_cct[i])):
        if(abs(delta_cct[i][j]) >= 25):
            up_dot_25 += 1
        if(abs(delta_cct[i][j]) >= 50):
            up_dot_50 += 1
        if(abs(delta_cct[i][j]) >= 100):
            up_dot_100 += 1
        if(abs(delta_cct[i][j]) >= 150):
            up_dot_150 += 1
        if(abs(delta_cct[i][j]) >= 200):
            up_dot_200 += 1
        if(abs(delta_cct[i][j]) >= 250):
            up_dot_250 += 1
        if(abs(delta_cct[i][j]) >= 300):
            up_dot_300 += 1
        if(abs(delta_cct[i][j]) >= 350):
            up_dot_350 += 1
        if(abs(delta_cct[i][j]) >= 400):
            up_dot_400 += 1
        if(abs(delta_cct[i][j]) >= 450):
            up_dot_450 += 1
        if(abs(delta_cct[i][j]) >= 500):
            up_dot_500 += 1
        
        # 편차 합 및 평균 저장
        hap += abs(delta_cct[i][j])
        avg = hap / len(delta_cct[i])

        # 편차 갯수들이 차지하는 비율 소수점 둘째짜리까지 계산
        percent_1 = round(up_dot_25 / len(cct[i]) * 100, 2)
        percent_2 = round(up_dot_50 / len(cct[i]) * 100, 2)
        percent_3 = round(up_dot_100 / len(cct[i]) * 100, 2)
        percent_4 = round(up_dot_150 / len(cct[i]) * 100, 2)
        percent_5 = round(up_dot_200 / len(cct[i]) * 100, 2)
        percent_6 = round(up_dot_250 / len(cct[i]) * 100, 2)
        percent_7 = round(up_dot_300 / len(cct[i]) * 100, 2)
        percent_8 = round(up_dot_350 / len(cct[i]) * 100, 2)
        percent_9 = round(up_dot_400 / len(cct[i]) * 100, 2)
        percent_10 = round(up_dot_450 / len(cct[i]) * 100, 2)
        percent_11 = round(up_dot_500 / len(cct[i]) * 100, 2)

    # 등급 선정 기준
    if(percent_1 > 30.0):
        grade += 1
    
    if(percent_2 > 20.0):
        grade += 1
    
    if(cct_hap[i] < 950000.0 or cct_hap[i] > 1050000.0):
        grade += 1
        
    if(hap > 15000.0):
        grade += 1
        
        # 편차의 절대값이 0.1 ~ 5이상인 경우의 갯수 저장
    up_dot_1 = 0
    up_dot_2 = 0
    up_dot_3 = 0
    up_dot_4 = 0
    up_dot_5 = 0
    up_dot_6 = 0
    up_dot_7 = 0
    up_dot_8 = 0
    up_dot_9 = 0
    up_dot_10 = 0
    up_dot_20 = 0
    up_dot_30 = 0
    up_dot_40 = 0
    up_dot_50_2 = 0

    # 위의 갯수가 하루 전체 데이터에서 차지하는 비율 저장
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_20 = 0
    percent_30 = 0
    percent_40 = 0
    percent_50_2 = 0

    # 편차의 합, 평균, 등급 저장
    hap_2 = 0
    avg_2 = 0
#     grade = 0

    for j in range(len(delta_swr[i])):
        if(abs(delta_swr[i][j]) >= 0.1):
            up_dot_1 += 1
        if(abs(delta_swr[i][j]) >= 0.2):
            up_dot_2 += 1
        if(abs(delta_swr[i][j]) >= 0.3):
            up_dot_3 += 1
        if(abs(delta_swr[i][j]) >= 0.4):
            up_dot_4 += 1
        if(abs(delta_swr[i][j]) >= 0.5):
            up_dot_5 += 1
        if(abs(delta_swr[i][j]) >= 0.6):
            up_dot_6 += 1
        if(abs(delta_swr[i][j]) >= 0.7):
            up_dot_7 += 1
        if(abs(delta_swr[i][j]) >= 0.8):
            up_dot_8 += 1
        if(abs(delta_swr[i][j]) >= 0.9):
            up_dot_9 += 1
        if(abs(delta_swr[i][j]) >= 1):
            up_dot_10 += 1
        if(abs(delta_swr[i][j] >= 2)):
            up_dot_20 += 1
        if(abs(delta_swr[i][j] >= 3)):
            up_dot_30 += 1
        if(abs(delta_swr[i][j] >= 4)):
            up_dot_40 += 1
        if(abs(delta_swr[i][j] >= 5)):
            up_dot_50_2 += 1

        hap_2 += abs(delta_swr[i][j])
        avg_2 = hap / len(delta_swr[i])

        percent_1 = round(up_dot_1 / len(swr[i]) * 100, 2);
        percent_2 = round(up_dot_2 / len(swr[i]) * 100, 2);
        percent_3 = round(up_dot_3 / len(swr[i]) * 100, 2);
        percent_4 = round(up_dot_4 / len(swr[i]) * 100, 2);
        percent_5 = round(up_dot_5 / len(swr[i]) * 100, 2);
        percent_6 = round(up_dot_6 / len(swr[i]) * 100, 2);
        percent_7 = round(up_dot_7 / len(swr[i]) * 100, 2);
        percent_8 = round(up_dot_8 / len(swr[i]) * 100, 2);
        percent_9 = round(up_dot_9 / len(swr[i]) * 100, 2);
        percent_10 = round(up_dot_10 / len(swr[i]) * 100, 2);
        percent_20 = round(up_dot_20 / len(swr[i]) * 100, 2);
        percent_30 = round(up_dot_30 / len(swr[i]) * 100, 2);
        percent_40 = round(up_dot_40 / len(swr[i]) * 100, 2);
        percent_50 = round(up_dot_50 / len(swr[i]) * 100, 2);

        # 등급 기준 1
#     if(swr_hap[i] <= 15500.0 or swr_hap[i] >= 16500.0):
#         grade += 1

#     # 등급 기준 2
#     # 편차의 절대값이 0.1 이상인 경우가 전체 데이터의 25% + 1, 30%를 넘을 경우 +1, 40%를 넘을 경우 +2, 50%를 넘길 경우 +3
#     if(percent_1 >= 25.0):
#         grade += 1

#     if(percent_1 >= 40.0):
#         grade += 1

#     # 등급 기준 3
#     # 편차의 절대값이 1 이상인 경우가 전체 데이터의 5%를 넘을 경우 +1, 10%를 넘을 경우 +2
#     if(percent_10 >= 10.0):
#         grade += 1
        
    wr.writerow([up_dot_25, up_dot_50, up_dot_100, up_dot_150, up_dot_200, up_dot_250, up_dot_300, up_dot_350, up_dot_400, up_dot_450, up_dot_500, up_dot_1, up_dot_2, up_dot_3, up_dot_4, up_dot_5, up_dot_6, up_dot_7, up_dot_8, up_dot_9, up_dot_10, up_dot_20, up_dot_30, up_dot_40, up_dot_50_2, grade])
w.close()

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
ohe = OneHotEncoder()
tf.set_random_seed(777)

import_data = np.genfromtxt('../data/cct_swr_calculation_rnn.csv', delimiter=',', dtype='float')

x_data = import_data[:, :25]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
x_data = x_data[:, :25]

y_data = []
test_y = []

for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][25])
    y_data.append(temp)

raw_y = y_data
    
y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

seq_length = 5
data_dim = 25
hidden_dim = 50
output_dim = 5
learning_rate = 0.001
iterations = 1001

dataX = []
dataY = []

for i in range(len(x_data) - seq_length):
    _x = x_data[i:i + seq_length]
    _y = y_data[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)

train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, output_dim])

W = tf.Variable(tf.random_normal([hidden_dim, output_dim]))
b = tf.Variable(tf.random_normal([output_dim]))

cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dim)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

tf.summary.scalar("accuracy", accuracy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter("./logs/rnn_logs", sess.graph)
merged = tf.summary.merge_all()

for i in range(iterations):
    _, step_cost = sess.run([optimizer, cost], feed_dict={X:trainX, Y:trainY})
    print("[step: {}] cost: {}".format(i, step_cost))
    summary, acc = sess.run([merged, accuracy], feed_dict={X:testX, Y:testY})
    writer.add_summary(summary, i)
    if (i % 10 == 0):
        print('학습 정확도:', sess.run(accuracy, feed_dict={X:trainX, Y:trainY}))
        print('테스트 정확도:', sess.run(accuracy, feed_dict={X:testX, Y:testY}))

# for i in range(len(trainX)):
#     a = sess.run(model, feed_dict={X:[trainX[i]]})
#     b = sess.run(Y, feed_dict={X:[trainX[i]]})
#     print(sess.run(tf.argmax(a, 1)), ", ", sess.run(tf.argmax(b, 1)))

# cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
# outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.nn.softmax)

# loss = tf.reduce_sum(tf.square(Y_pred - Y))
# optimizer = tf.train.AdamOptimizer(learning_rate)
# train = optimizer.minimize(loss)

# targets = tf.placeholder(tf.float32, [None, 5])
# predictions = tf.placeholder(tf.float32, [None, 5])
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for i in range(iterations):
#         _, step_loss = sess.run([train, loss], feed_dict={X:trainX, Y:trainY})
#         print("[step: {}] loss: {}".format(i, step_loss))
        
#     test_predict = sess.run(Y_pred, feed_dict={X:testX})
#     rmse_val = sess.run(rmse, feed_dict={targets:testY, predictions:test_predict})
#     print("RMSE:{}".format(rmse_val))
    
#     print(sess.run(tf.argmax(test_predict, 1)))
#     print(len(dataX), len(trainX), len(testX))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

[step: 0] cost: 1.8057533502578735
학습 정확도: 0.16780046
테스트 정확도: 0.16842106
[step: 1] cost: 1.740919828414917
[step: 2] cost: 1.6917649507522583
[step: 3] cost: 1.656561017036438
[step: 4] cost: 1.633130669593811
[step: 5] cost: 1.6187456846237183
[step: 6] cost: 1.6101666688919067
[step: 7] cost: 1.6043492555618286
[step: 8] cost: 1.5993471145629883
[step: 9] cost: 1.5944291353225708
[step: 10] cost: 1.5895594358444214
학습 정확도: 0.34920636
테스트 정확도: 0.3105263
[step: 11] cost: 1.5848288536071777
[step: 12] cost: 1.5801514387130737
[step: 13] cost: 1.575265884399414
[step: 14] cost: 1.5699036121368408
[step: 15] cost: 1.5639374256134033
[step: 16] cost: 1.5574254989624023
[step: 17] cost: 1.5505715608596802
[step: 18] cost: 1.543650507926941
[step: 19] cost: 1.5369367599487305
[step: 20] cost: 1.530651569366455
학습 정확도: 0.35827664
테스트 정확도: 0.32105264
[step: 21] cost: 1.5249301195144653
[step: 22] cost: 1.5198203325271606
[step: 23] cost: 1.5152982473373413
[step: 24] cost: 1.5112965106964111


[step: 213] cost: 1.1642495393753052
[step: 214] cost: 1.1599509716033936
[step: 215] cost: 1.1593098640441895
[step: 216] cost: 1.1592100858688354
[step: 217] cost: 1.1551833152770996
[step: 218] cost: 1.1524972915649414
[step: 219] cost: 1.1522608995437622
[step: 220] cost: 1.1498545408248901
학습 정확도: 0.5419501
테스트 정확도: 0.22105263
[step: 221] cost: 1.1467243432998657
[step: 222] cost: 1.1455084085464478
[step: 223] cost: 1.1440818309783936
[step: 224] cost: 1.1412978172302246
[step: 225] cost: 1.1392238140106201
[step: 226] cost: 1.1380139589309692
[step: 227] cost: 1.1359022855758667
[step: 228] cost: 1.1333342790603638
[step: 229] cost: 1.1317341327667236
[step: 230] cost: 1.1301796436309814
학습 정확도: 0.5442177
테스트 정확도: 0.22105263
[step: 231] cost: 1.1277167797088623
[step: 232] cost: 1.1255062818527222
[step: 233] cost: 1.123914122581482
[step: 234] cost: 1.1220029592514038
[step: 235] cost: 1.119668960571289
[step: 236] cost: 1.1175129413604736
[step: 237] cost: 1.1157411336898804
[

[step: 418] cost: 0.7582892179489136
[step: 419] cost: 0.7565337419509888
[step: 420] cost: 0.7546448707580566
학습 정확도: 0.73015875
테스트 정확도: 0.21578947
[step: 421] cost: 0.7527971267700195
[step: 422] cost: 0.7509718537330627
[step: 423] cost: 0.7491216063499451
[step: 424] cost: 0.7472283840179443
[step: 425] cost: 0.7453820109367371
[step: 426] cost: 0.7435087561607361
[step: 427] cost: 0.7416317462921143
[step: 428] cost: 0.7397322058677673
[step: 429] cost: 0.737845778465271
[step: 430] cost: 0.7359555959701538
학습 정확도: 0.7346939
테스트 정확도: 0.21578947
[step: 431] cost: 0.7340583801269531
[step: 432] cost: 0.7321305871009827
[step: 433] cost: 0.7302212715148926
[step: 434] cost: 0.7283118963241577
[step: 435] cost: 0.7263898253440857
[step: 436] cost: 0.7244478464126587
[step: 437] cost: 0.7225141525268555
[step: 438] cost: 0.7205742001533508
[step: 439] cost: 0.7186343669891357
[step: 440] cost: 0.7166869640350342
학습 정확도: 0.7414966
테스트 정확도: 0.2263158
[step: 441] cost: 0.7147334218025208

[step: 621] cost: 0.4158991575241089
[step: 622] cost: 0.41444292664527893
[step: 623] cost: 0.41297370195388794
[step: 624] cost: 0.4115811288356781
[step: 625] cost: 0.41023892164230347
[step: 626] cost: 0.40887683629989624
[step: 627] cost: 0.4074777066707611
[step: 628] cost: 0.40606042742729187
[step: 629] cost: 0.4046502113342285
[step: 630] cost: 0.4032628536224365
학습 정확도: 0.8707483
테스트 정확도: 0.19473684
[step: 631] cost: 0.4018973112106323
[step: 632] cost: 0.40053197741508484
[step: 633] cost: 0.39914172887802124
[step: 634] cost: 0.3977278769016266
[step: 635] cost: 0.39632248878479004
[step: 636] cost: 0.39494654536247253
[step: 637] cost: 0.3935849666595459
[step: 638] cost: 0.39220893383026123
[step: 639] cost: 0.39081159234046936
[step: 640] cost: 0.38941097259521484
학습 정확도: 0.877551
테스트 정확도: 0.19473684
[step: 641] cost: 0.3880223333835602
[step: 642] cost: 0.38664379715919495
[step: 643] cost: 0.38526639342308044
[step: 644] cost: 0.38388654589653015
[step: 645] cost: 0.38

[step: 825] cost: 0.18757769465446472
[step: 826] cost: 0.1868007630109787
[step: 827] cost: 0.18602673709392548
[step: 828] cost: 0.18525557219982147
[step: 829] cost: 0.18448728322982788
[step: 830] cost: 0.18372197449207306
학습 정확도: 0.95464855
테스트 정확도: 0.2
[step: 831] cost: 0.18295952677726746
[step: 832] cost: 0.18219992518424988
[step: 833] cost: 0.18144327402114868
[step: 834] cost: 0.18068961799144745
[step: 835] cost: 0.17993871867656708
[step: 836] cost: 0.17919081449508667
[step: 837] cost: 0.17844584584236145
[step: 838] cost: 0.17770376801490784
[step: 839] cost: 0.17696459591388702
[step: 840] cost: 0.1762283891439438
학습 정확도: 0.95464855
테스트 정확도: 0.2
[step: 841] cost: 0.17549508810043335
[step: 842] cost: 0.17476478219032288
[step: 843] cost: 0.17403730750083923
[step: 844] cost: 0.1733127385377884
[step: 845] cost: 0.17259114980697632
[step: 846] cost: 0.17187246680259705
[step: 847] cost: 0.17115676403045654
[step: 848] cost: 0.17044392228126526
[step: 849] cost: 0.1697340

In [2]:
seq_length = 5
RNN

seq_length = 4
RNN
230
37.37

LSTM
180
36.12

seq_length = 3
460
37.17

404
36.64

NameError: name 'RNN' is not defined